# AI SQL
AISQL delivers an easy-to-use composable query language that handles both text and multimodal data through familiar SQL syntax. Our high-performance batch engine processes queries faster through intelligent query optimization, while delivering lower costs than traditional AI solutions. Natively ingest structured and unstructured data into unified multimodal tables, enabling comprehensive insights through familiar SQL analytics for all your data.

## AI SQL Benefits:
### 1) Easy-to-use SQL syntax to build AI pipelines without complex coding
### 2) High-performance processing across all modalities (text, image, audio)
### 3) Lower cost batch processing through optimized architecture to support faster and larger batch jobs.


## Three Major function calls
### AI_FILTER
    AI-powered SQL operator for semantic filtering. You can use the same syntax on a single table (for filtering) or join multiple tables together upon a semantic relationship. 

### AI_AGG/AI_SUMMARIZE_AGG
    Advanced functionalities provide unmatched comprehensive analysis across multiple data entries with a user prompt. This method overcomes contextual window constraints, facilitating extensive multi-record examination with no comparable solutions in the market.

### AI_CLASSIFY
    Functions for easy classification across multi-modal data

# Resturant Review Batch Unstructured Processing

Snowflake's AI powered functions enable column level operations with LLM like traditional database operators. 

Image Processing
1. Load data into FILE column
2. Using COMPLETE(IMAGE) Function to add descriptions for each image
3. Using AI_FILTER(IMAGE) and AI_CLASSIFY(IMAGE) to filter/classify accordingly.
4. Using AI_FILTER() to join food IMAGE with corresponding text from the food menu.

With this import we can run streamlit, pandas, and the snowbook extras python script we imported.  The snowbook extras allows us to view the images in table format below.

In [ ]:
import streamlit as st
import pandas as pd

In [ ]:
ls @SNOWFLAKE_INTELLIGENCE.PUBLIC.FOODIMAGES;

In [ ]:
USE DATABASE SNOWFLAKE_INTELLIGENCE;
USE SCHEMA SNOWFLAKE_INTELLIGENCE.PUBLIC;

create or replace table img_tbl as
select to_file(file_url) img_file, * from directory(@FOODIMAGES);

In [ ]:
SELECT * FROM img_tbl LIMIT 7;

In [ ]:
desc table img_tbl;

Below we can see the file images using our snowbooks_extras.py helper file and see the data we are working with.  You can click the image below to see a larger image

In [ ]:
select img_file, file_url, last_modified from img_tbl LIMIT 5;

Below we use a helper function in the cortex complete to prompt to read the image and write an advertisement for the dish in the image.

In [ ]:
-- test complete(image)

CREATE OR REPLACE TABLE IMAGE_TEXT_TBL AS
select 
    img_file,
    snowflake.cortex.complete('pixtral-large', prompt('Write me an advertisement tagline for this dish: {0}', img_file)) AS descrip,
    file_url,
    last_modified
from img_tbl;

SELECT * FROM IMAGE_TEXT_TBL;

Now we will run a new [AI_FILTER](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_filter) to filter out the images and look only for those that are desserts from our list of images. We want to user AI_FILTER with a prompt to help us tell if the image in the picture is a dessert.  Please edit the code below in the where statement properly use the AI_FILTER

In [ ]:
select * 
from IMAGE_TEXT_TBL
where AI_FILTER(PROMPT('find out if this: {0} is a dessert', descrip));

In [ ]:
/*select * 
from (
    select 
        img_file,
        snowflake.cortex.complete('pixtral-large', prompt('Write me an advertisement tagline for this dish: {0}', img_file)) AS descrip,
        file_url,
        last_modified
        from img_tbl
    )
where AI_FILTER(PROMPT('find out if this: {0} is a dessert', descrip));
*/
SELECT CURRENT_ROLE();

Let's further classify these images using [AI_CLASSIFY](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_classify).  We specify what classifications we want in the prompt and ask the GenAI model to break up the images into those classes

In [ ]:
-- Notice we are passing the image
WITH food_pictures AS (
  SELECT
      *
  FROM IMAGE_TEXT_TBL
)
SELECT
    *,
    AI_CLASSIFY(IMG_FILE, ['dessert', 'drink', 'main dish', 'side dish']):labels AS classification
FROM food_pictures;

Now we can filter those reviews using [AI_FILTER](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_filter-snowflake-cortex) on the context we added previously and look for reviews that mentioned a special occasion like a birthday or other